<a href="https://colab.research.google.com/github/olodenm/info_2022_olo/blob/Netology_HW/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%C2%AB%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8_%D0%B8_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

1.   оценка 2 и ниже — низкий рейтинг;
2.   оценка 4 и ниже — средний рейтинг;
3. оценка 4.5 и 5 — высокий рейтинг.


Результат классификации запишите в столбец class.

In [ ]:
# Загружаем датафрейм с таблицей фильмов и выводим первые 5 значений.
movie = pd.read_csv('/content/movies.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Загружаем датафрейм с таблицей рейтингов и выводим первые 5 значений.
ratings = pd.read_csv('/content/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
# методом pd.merge соединяем две таблицы по ключу 'movieId',
# левый джойн для того чтобы в результат попали все фильмы
movie_ratings = pd.merge(movie, ratings, on='movieId', how='left')
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08


In [ ]:
# Группируем данные по 'movieId' и вычисляем средний рейтинг для каждого фильма
average_ratings = movie_ratings.groupby('movieId')['rating'].mean()

In [ ]:
# Добавляем средний рейтинг в исходный DataFrame
movie_ratings = movie_ratings.merge(average_ratings, on='movieId')
movie_ratings.head()

,movieId,title,genres,userId,rating_x,timestamp,rating_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08,3.87247
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08,3.87247
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09,3.87247
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08,3.87247
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08,3.87247


In [ ]:
# Переименновываю столбцы
movie_ratings.columns = ['id', 'title', 'genres', 'userid', 'rating', 'timestamp', 'rating_agv']
movie_ratings.head()

,id,title,genres,userid,rating,timestamp,rating_agv
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08,3.87247
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08,3.87247
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09,3.87247
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08,3.87247
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08,3.87247


In [ ]:
def classify_movies(movie_ratings):
    """
    Классифицирует фильмы по рейтингу с учетом дополнительных условий, используя between.
    Rating фильмов в диапазоне (> 4.0 и < 4.5) и (>4.5  и < 5.0) = отсутствует
    Args:
        movie_ratings: DataFrame с данными о фильмах.

    Returns:
        Замененный столбец class с результатами классификации.
    """

    movie_ratings['class'] = 'рейтинг отсутствует'
    movie_ratings.loc[movie_ratings['rating_agv'] <= 2.0, 'class'] = 'низкий'
    movie_ratings.loc[movie_ratings['rating_agv'].between(2.1, 4.0), 'class'] = 'средний'
    movie_ratings.loc[movie_ratings['rating_agv'] == 4.5, 'class'] = 'высокий'
    movie_ratings.loc[movie_ratings['rating_agv'] == 5.0, 'class'] = 'высокий'

    return movie_ratings

movie_ratings = classify_movies(movie_ratings)


In [ ]:
movie_ratings.head()

,id,title,genres,userid,rating,timestamp,rating_agv,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08,3.87247,средний
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08,3.87247,средний
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09,3.87247,средний
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08,3.87247,средний
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08,3.87247,средний


Задание 2

---


Используйте файл [keywords.csv](/content/keywords.csv).

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [ ]:
# Загружаем датафрейм с таблицей ключевые слова и выводим первые 5 значений.
keywords = pd.read_csv('/content/keywords.csv')
keywords

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [ ]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [ ]:
# Функция для определения региона
def classify_region(keyword):
    for region, cities in geo_data.items():
        for city in cities:
            if city in keyword:
                return region
    return 'undefined'

# Применяем функцию к столбцу 'keyword' и создаем новый столбец 'region'
keywords['region'] = keywords['keyword'].apply(classify_region)

keywords

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
99995,эльдорадо старый оскол,3705,undefined
99996,frigate для firefox,3630,undefined
99997,укрсиб,3630,undefined
99998,погода в ялте на неделю,3688,undefined


Задание 3 (бонусное)


---


Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010 года.

1. **В переменную years запишите список из всех годов с 1950 по 2010 года.**
2. **Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:**

      1.   для каждой строки пройдите по всем годам списка years;
      если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
      2.   если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
        Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

3. **Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.**

In [ ]:
# В переменную years записываем list из всех годов с 1950 по 2010 года.
years = list(range(1950, 2010))

def  production_year(movie_title):
    """
    Функция, которая извлекает год выпуска фильма из его названия.

    Args:
        movie_title (str): Название фильма.

    Returns:
        int: Год выпуска фильма, если год найден в названии, иначе 1900.
    """
    for i in years:
        if str(i) in movie_title:
            return i
    return 1900

# Записываем год выпуска фильма в новый столбец ‘year’.
movie_ratings['year'] = movie_ratings['title'].apply(production_year)

# Считаем средний рейтинг для каждого года выпуска фильма
ratings_by_year = movie_ratings.groupby('year')['rating'].mean()

# Сортируем результат по убыванию рейтинга
ratings_by_year.sort_values(ascending=True, inplace=True)

ratings_by_year

year
2000    3.356104
1997    3.415764
1996    3.426600
2003    3.444777
2005    3.448434
          ...   
1951    3.983539
1954    3.994220
1952    4.000000
1972    4.011136
1957    4.014241
Name: rating, Length: 61, dtype: float64